In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
from scipy.optimize import curve_fit
from scipy import ndimage
from astropy.modeling import models, fitting
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numba
from numba import jit
import pandas as pd
from colossus.cosmology import cosmology

#plt.rcParams['text.usetex'] = True

%matplotlib inline

# Different order SZ spectrum with cluster parameters : T_e = 5 keV, tau = 0.01 and peculiar velocity  = 300 km/s

In [ ]:
def calculate_kSZ_SED(nu, h, k, t_cmb):
    
    I_o = 270.33
    ksz_SED = []
    
    for i in range(len(nu)):
        
        x = (h * nu[i] * 1e9 )/(k * t_cmb)
        spec = -(I_o * (np.power(x,4) * np.exp(x))/((np.exp(x) - 1) ** 2) * v * mu * tau)
        ksz_SED.append(spec)
        
    return ksz_SED


In [ ]:
def calculate_tsz_SED(nu, h, k, t_cmb):
    
    tsz_SED = []
    
    for i in range(len(nu)):
        
        x = (h * nu[i] * 1e9 )/(k * t_cmb)
        F = x * (1 / np.tanh(x / 2))
        spec = (-4 + F) 
        tsz_SED.append(spec)
        
    return tsz_SED



In [ ]:
def calculate_rksz_SED(nu, h, k, mu, t_cmb):
    
    rksz_SED = []
    
    for i in range(len(nu)):
        
        x = (h * nu[i] * 1e9)/(k * t_cmb)
        F = x * (1 / np.tanh(x / 2))
        spec = (-1 - mu ** 2 + ( (3 + 11 * mu ** 2) / (20)) * F) 
        rksz_SED.append(spec)
        
    return rksz_SED



In [ ]:
def calculate_tksz_SED(nu, h, k, t_cmb):
     
    tksz_SED = []
    
    for i in range(len(nu)):
        
        x = (h * nu[i] *1e9 )/(k * t_cmb)
        F = x * (1 / np.tanh(x / 2))
        G = x / np.sinh(x / 2)
        spec =  (10 - 9.4 * F + 0.7 * ( 2 * np.power(F, 2) + np.power(G, 2)) ) 
        tksz_SED.append(spec)
        
    return tksz_SED



In [ ]:
def calculate_rtsz_SED(nu, h, k, t_cmb):
    
    rtsz_SED = []
    
    for i in range(len(nu)):
        
        x = (h * nu[i] * 1e9)/(k * t_cmb)
        F = x * (1 / np.tanh(x / 2))
        G = x / np.sinh(x / 2)
        spec = (-10 + 23.5 * F - 8.4 * np.power(F, 2) + 0.7 * np.power(F, 3) + 1.4 * np.power(G, 2) * (-3 + F)) 
        rtsz_SED.append(spec)
        
    return rtsz_SED

# CMB temperature map from TT power spectrum

In [ ]:
ell, DlTT = np.loadtxt("CAMB_fiducial_cosmo_scalCls.txt", usecols=(0, 1), unpack=True) 


#Setting size of the map

N = 256      # number of pixels in a linear dimension
            
pix_size  = 1.1 # size of a pixel in arcminutes

## Setting up the map plots

c_min = -400              # minimum for color bar
c_max = 400               # maximum for color bar
X_width = N * pix_size/60.  # horizontal map width in degrees
Y_width = N * pix_size/60.  # vertical map width in degrees

In [ ]:

def make_CMB_T_map(N,pix_size,ell,DlTT):  # makes a realization of a simulated CMB sky map given an input D_l(TT) as a function of l
     
# convert D_l to C_l  

    ClTT = (DlTT * 2 * np.pi) / (ell * (ell + 1)) 
    ClTT[0] = 0.                          # set the monopole and the dipole of the C_l spectrum to zero
    ClTT[1] = 0.
   
    
# make a 2D real space coordinate system

    onesvec = np.ones(N)
    inds  = (np.arange(N) + .5 - N/2.) / (N-1.) # create an array of size N between -0.5 and +0.5
   
# compute the outer product matrix: X[i, j] = onesvec[i] * inds[j] for i,j 
# in range(N), which is just N rows copies of inds - for the x dimension
    X = np.outer(onesvec,inds) 

# compute the transpose for the y dimension
    Y = np.transpose(X)

# radial component 
    R = np.sqrt( X**2. + Y**2.)
   

# 2D CMB power spectrum

    pix_to_rad = (pix_size/60. * np.pi/180.) 
    ell_scale_factor = 2. * np.pi /pix_to_rad 
    ell2d = R * ell_scale_factor            # making a fourier space analogue to the real space R vector
    ClTT_expanded = np.zeros(int(ell2d.max())+1) 
    ClTT_expanded[0:(ClTT.size)] = ClTT     # fill in the Cls until the max of the ClTT vector

# the 2D Cl spectrum is defined on the multiple vector set by the pixel scale
    CLTT2d = ClTT_expanded[ell2d.astype(int)] 
#plt.imshow(np.log(CLTT2d))

#realization of the CMB with the given power spectrum in real space
    random_array_for_T = np.random.normal(0,1,(N,N))
    FT_random_array_for_T = np.fft.fft2(random_array_for_T)   # take FFT since we are in Fourier space 
    FT_2d = np.sqrt(CLTT2d) * FT_random_array_for_T 

# move back from ell space to real space
    CMB_T = np.fft.ifft2(np.fft.fftshift(FT_2d))

# move back to pixel space for the map
    CMB_T = CMB_T/(pix_size /60.* np.pi/180.)
    
# only plot the real component
    CMB_T = np.real(CMB_T)

    return(CMB_T)


In [ ]:

def plot_map(x,y,map_title,map_plot,cbar_label, N, pix_size):
    
    plt.gcf().set_size_inches(x,y)

    plt.title(map_title, fontsize=15, fontweight='bold')
    image = plt.imshow(map_plot, cmap = 'jet', origin = 'lower')
    cb = plt.colorbar(image)
    cb.set_label(label = cbar_label, fontsize=15, fontweight='bold')
    cb.ax.tick_params(labelsize=15)
    
    X_width = N * pix_size/60.  # horizontal map width in degrees
    Y_width = N * pix_size/60.  # vertical map width in degrees
    

    image.set_extent([0,X_width,0,Y_width])
    plt.ylabel('angle $[^\circ]$', fontsize=15)
    plt.xlabel('angle $[^\circ]$', fontsize=15) 
    ax = plt.gca()


    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    ax.spines['top'].set_linewidth(0.8)
    ax.spines['right'].set_linewidth(0.8)
    ax.spines['bottom'].set_linewidth(0.8)
    ax.spines['left'].set_linewidth(0.8)
#     plt.tight_layout()
    
    

    

# Polyfitting CMB Map

In [ ]:
def poly_filtering(input_map, order, N):
    
    onesvec = np.ones(N)
    inds  = (np.arange(N) + .5 - N/2.) / (N-1.) 
   
    X = np.outer(onesvec,inds) 
    Y = np.transpose(X)

    poly_init = models.Polynomial2D(order)
    fit_poly = fitting.LinearLSQFitter()#LevMarLSQFitter()
    fitted_poly = fit_poly(poly_init, X, Y, input_map)
       
    filtered_map = input_map - fitted_poly(X, Y)
    
    return filtered_map



# White noise map

In [ ]:
noises_planck = np.array([7.73, 3.34, 4.65, 15.6, 80.6, 1920]) # reduced by factor of 10
fwhm_planck = np.array([9.68, 7.30, 5.02, 4.94, 4.83, 4.64])

noises_CORE = np.array([7.1, 5.1, 3.6, 3.8, 11.1, 45.9, 358.3])
fwhm_CORE = np.array([15.39, 12.08, 7.68, 5.23, 3.49, 2.65, 1.98])

noises_cmbs4 = np.array([1.5, 1.5, 4.8, 11.5])
fwhm_cmbs4 = np.array([2.2, 1.4, 1.0, 0.9])

In [ ]:
noise = 2 #7.73, 3.34, 4.65, 15.6, 80.6, 1920 in muK arcmin

def create_noise_map(N,noise,pix_size):
    
    mu = 0 
    
    gaussian_random_noise_generation = mu + np.random.randn(N,N) * noise /pix_size
   
    return gaussian_random_noise_generation


# Simulated y signal map from isothermal beta model

# All channel noise maps of Planck, CORE and CMBS4

In [ ]:
noises_planck = np.array([7.73, 3.34, 4.65, 15.6, 80.6, 1920]) # reduced by factor of 10
fwhm_planck = np.array([9.68, 7.30, 5.02, 4.94, 4.83, 4.64])

noises_CORE = np.array([7.1, 5.1, 3.6, 3.8, 11.1, 45.9, 358.3])
fwhm_CORE = np.array([15.39, 12.08, 7.68, 5.23, 3.49, 2.65, 1.98])

noises_cmbs4 = np.array([1.5, 1.5, 4.8, 11.5])
fwhm_cmbs4 = np.array([2.2, 1.4, 1.0, 0.9])

In [ ]:
def all_channel_noise_maps( N, z, pix_size, noises):
    
    total_noise_maps = []
    
    
    for j in range(len(z)):
        
        each_noise_maps = []    # same size of pixels at differenet frequencies

        for i in range(len(noises)):
            
            mu = 0
        
            noise_map_gen = mu + np.random.randn(N,N) * noises[i] /pix_size[j]
            each_noise_maps.append(noise_map_gen)
            
        total_noise_maps.append(each_noise_maps)
      
    return np.array(total_noise_maps)


In [ ]:


def cmb_stacking(no_realisations):
    
    cmb_maps = []

    for i in range(no_realisations):
         
         cmb_temp_map = make_CMB_T_map(N,pix_size,ell,DlTT)
         cmb_maps.append(cmb_temp_map)
    
    stacked_cmb = np.sum(cmb_maps, axis = 0)/no_realisations
    
    return stacked_cmb


In [ ]:


def cmb_filtered_stacking(no_realisations):
    
    cmb_maps = []

    for i in range(no_realisations):
    
         cmb_temp_map = make_CMB_T_map(N,pix_size,ell,DlTT)
         cmb_map_filtered = poly_filtering(cmb_temp_map, 2, N)
         cmb_maps.append(cmb_map_filtered)
    
    stacked_cmb = np.sum(cmb_maps, axis = 0)/no_realisations
    
    return stacked_cmb


# Stacked noise maps

In [ ]:

def noise_map_stacking(no_realisations, noises):
    
    noise_maps = []

    for i in range(len(noises)):
        
        f_noise_maps = []
        
        for j in range(no_realisations):
        
             all_noise = create_noise_map(N, noises[i], pix_size)
             f_noise_maps.append(all_noise)
            
        noise_maps.append(f_noise_maps)
        
    stacked_noise_maps = []
        
    for k in range(len(noises)):
    
        stacked_noise_map = np.sum(noise_maps[k], axis = 0)/no_realisations
        stacked_noise_maps.append(stacked_noise_map)
    
    return stacked_noise_maps


# Smoothing maps

In [ ]:
def calculate_smoothed_maps(z, pix_size, fwhm_beam, total_maps):
    
    total_maps_smoothed = []
    
    for j in range(len(z)):
        
        maps_smoothed = []
    
        each_map = total_maps[j]
    
        for i in range(len(fwhm_beam)):
        
            sigma_beam = fwhm_beam[i] /(np.sqrt(8*np.log(2)))/pix_size[j]
        
            map_smoothing = ndimage.gaussian_filter(each_map[i], 
                    sigma = sigma_beam, order = 0, mode = 'reflect', truncate = 10)
        
            sigma_beam_cs = (np.sqrt(fwhm_beam[0]**2 - fwhm_beam[i]**2))/(np.sqrt(8*np.log(2)))/pix_size[j]
        
            map_smoothing_cs = ndimage.gaussian_filter(map_smoothing, 
                    sigma = sigma_beam_cs, order = 0, mode = 'reflect', truncate = 10)
        
            maps_smoothed.append(map_smoothing_cs)
            
        total_maps_smoothed.append(maps_smoothed)
        
    return np.array(total_maps_smoothed)

In [ ]:
def calculate_smoothed_maps_freq_independent(z, pix_size, fwhm_beam, total_maps):
    
    total_maps_smoothed = []
    
    for j in range(len(z)):
        

            sigma_beam = 2.2 /(np.sqrt(8*np.log(2)))/pix_size[j]
        
            maps_smoothed = ndimage.gaussian_filter(total_maps[j], 
                    sigma = sigma_beam, order = 0, mode = 'reflect', truncate = 10)
        
            
            
            total_maps_smoothed.append(maps_smoothed)
        
    return np.array(total_maps_smoothed)

# Simulating non relativistic SZ signals

## Gaussian velocity distribution

In [ ]:

def create_gaussian_vel(realisations, sd_vel):
    
    mu = 0 
    gaussian_vel_generation = mu + np.random.randn(realisations) * sd_vel
    
    return gaussian_vel_generation



# Simulated kSZ signal map 

# $\Delta T_{ksz} = \tau \mu \frac{V}{c} T_{CMB}$

$ \tau $ is optical depth

ksz map = $\tau (p) \mu \frac{V}{c} T_{CMB} \times 10^6$



In [ ]:
def calculate_ksz_maps(tau_map,z, v, mu, t_cmb):
    
    ksz_maps = [] 

    for i in range(len(z)):
    
         ksz_map = tau_map[i] * v[i] * mu * t_cmb * 1e6
         ksz_maps.append(ksz_map)
    
    return ksz_maps


# CMB + kSZ map

In [ ]:
def calculate_cmb_ksz_map(tau_map,z, v, N, r_500, ell, DlTT, mu, t_cmb ):
    
    cmb_ksz_maps = []
    
    for i in range (len(z)):

        ksz_maps = calculate_ksz_maps(tau_map,z, v, mu, t_cmb)[i]
        cmb_map =  make_CMB_T_map(N,r_500[i],ell,DlTT)             # different realisations
        cmb_ksz_map = ksz_maps + cmb_map
        cmb_ksz_maps.append(cmb_ksz_map)
    
    return cmb_ksz_maps


In [ ]:
def calculate_cmb_ksz_stacked_map(no_realisations, vel ):
    
    cmb_ksz_maps = []

    for i in range(no_realisations):
    
         cmb_temp_map = make_CMB_T_map(N,pix_size,ell,DlTT)
         ksz_map = (beta_map * (511/5) * vel[i] * mu * t_cmb * 1e6)
         cmb_ksz_map = ksz_map + cmb_temp_map
         cmb_ksz_maps.append(cmb_ksz_map)
    
    stacked_cmb_ksz = np.sum(cmb_ksz_maps, axis = 0)/no_realisations
    
    return stacked_cmb_ksz


# Filtered kSZ+CMB

In [ ]:

def calculate_cmb_ksz_filtered_stacked_map(no_realisations, vel ):
    
    cmb_ksz_filtered_maps = []

    for i in range(no_realisations):
    
         cmb_temp_map = make_CMB_T_map(N,pix_size,ell,DlTT)
         cmb_ksz_map = (beta_map * (511/5) * vel[i] * mu * t_cmb * 1e6) + cmb_temp_map
         cmb_ksz_filtered = poly_filtering(cmb_ksz_map, 2, N)
         cmb_ksz_filtered_maps.append(cmb_ksz_filtered)
    
    stacked_cmb_ksz_filtered = np.sum(cmb_ksz_filtered_maps, axis = 0)/no_realisations
    
    return stacked_cmb_ksz_filtered


# (CMB+kSZ) + channel noises at CMBS4 frequencies


In [ ]:

########## Simulated CMB+kSZ + channel noises at diffrent frequencies ##########

def calculate_total_cmb_ksz_map( N, pix_size, nu, cmb_ksz_signal , noise_map):
    
    total_cmb_ksz_maps = []

    for i in range(len(nu)):
        
        total_cmb_ksz_map =  cmb_ksz_signal + noise_map[i]  
        total_cmb_ksz_maps.append(total_cmb_ksz_map)
        
    return total_cmb_ksz_maps



########## Simulated filtered CMB+kSZ + channel noises at diffrent frequencies ##########


def calculate_total_filtered_cmb_ksz_map( N, pix_size, nu, input_map, noise_map):
    
    total_filtered_cmb_ksz_maps = []

    for i in range(len(nu)):
        
        total_filtered_cmb_ksz_map =  input_map + noise_map[i]  
        total_filtered_cmb_ksz_maps.append(total_filtered_cmb_ksz_map)
        
    return total_filtered_cmb_ksz_maps


# tSZ Map at different CORE and CMBS4 frequencies 

# $\Delta T = \tau \frac{kT_e}{m_ec^2}f(x)T_{CMB}$
$f(x) = xcoth(x/2)-4$

$x = h\nu/k_BT_{CMB}$

tSZ temp. map at different frequencies :

$\Delta T = beta(p)f(x)T_{CMB}\times10^6$

In [ ]:
def calculate_tsz_map(nu, z, tsz_spec, y_map, t_cmb):
    
    total_tsz_maps = []
    
    for j in range(len(z)):
        
        tsz_maps = []
        picked_y_map = y_map[j]
        
        for i in range(len(nu)):
         
            tsz_map = tsz_spec[i] * t_cmb * 1e6 * picked_y_map
            tsz_maps.append(tsz_map)
            
        total_tsz_maps.append(tsz_maps)
    
    return np.array(total_tsz_maps)


# tSZ + Noise + CMB at CMBS4 and CORE

In [ ]:
def calculate_total_tsz_map( N, pix_size, nu, cmb_map, tsz_signal, noise_map):
    
    total_tsz_maps = []

    for i in range(len(nu)):
        
        total_tsz_map =  tsz_signal[i] + noise_map[i] + cmb_map 
        total_tsz_maps.append(total_tsz_map)
        
    return total_tsz_maps



# Simulating higher order SZ signals

# rkSZ Map at different CMB S4 and CORE frequencies

In [ ]:
def calculate_rksz_map(nu, z, rksz_spec, tau_map, v, t_cmb):
    
    total_rksz_maps = []
    
    for j in range(len(z)):
        
        rksz_maps = []
        picked_tau_map = tau_map[j]
        
        for i in range(len(nu)):
         
            rksz_map = rksz_spec[i]* (v[j]**2) * t_cmb * 1e6 * picked_tau_map
            rksz_maps.append(rksz_map)
            
        total_rksz_maps.append(rksz_maps)
    
    return np.array(total_rksz_maps)

# Stacked rkSZ maps at different CORE and CMBS4 frequencies

In [ ]:
def calculate_input_rksz_signal(z, v, tau_map):
    
    signals = []
    
    for i in range(len(z)):
        
        input_rksz = (v[i]**2) * tau_map[i]
        signals.append(input_rksz)
        
    return np.array(signals)

# rkSZ signal + noise + CMB map at CMBS4 frequencies

In [ ]:
def calculate_total_rksz_map( N, pix_size, nu, cmb_map, rksz_signal, noise_map):
    
    total_rksz_maps = []

    for i in range(len(nu)):
        
        total_rksz_map =  rksz_signal[i] + noise_map[i] + cmb_map 
        total_rksz_maps.append(total_rksz_map)
        
    return total_rksz_maps


# tkSZ Map at different CMB S4 and CORE frequencies                                                                   

In [ ]:
def calculate_input_tksz_signal(z,v,mu,y_map):
    
    signals = []
    
    for i in range(len(z)):
        
        input_tksz = v[i]*mu * y_map[i]
        signals.append(input_tksz)
        
    return np.array(signals)

In [ ]:
def calculate_tksz_map(nu, z, tksz_spec, y_map, v, mu, t_cmb):
    
    total_tksz_maps = []
    
    for j in range(len(z)):
        
        tksz_maps = []
        picked_y_map = y_map[j]
        
        for i in range(len(nu)):
         
            tksz_map = tksz_spec[i]* v[j] * mu * picked_y_map * t_cmb * 1e6
            tksz_maps.append(tksz_map)
            
        total_tksz_maps.append(tksz_maps)
    
    return np.array(total_tksz_maps)

# Stacked tksz signal at CORE and CMBS4 frequencies

# tkSZ + noise + CMB map for CMB S4 and CORE frequencies

In [ ]:
def calculate_total_tksz_map( N, pix_size, nu, cmb_map, tksz_signal, noise_map):
    
    total_tksz_maps = []

    for i in range(len(nu)):
        
        total_tksz_map = tksz_signal[i] + noise_map[i] + cmb_map 
        total_tksz_maps.append(total_tksz_map)
        
    return total_tksz_maps



# rtSZ Map at different Planck HFI and CORE frequencies

In [ ]:
def calculate_input_rtsz_signal(z,y_map, t_e):
    
    signals = []
    
    for i in range(len(z)):
        
        input_rtsz = y_map[i] * (t_e[i]/511)
        signals.append(input_rtsz)
        
    return np.array(signals)

In [ ]:
def calculate_rtsz_map(nu, z, rtsz_spec, y_map, mu, t_e, t_cmb):
    
    total_rtsz_maps = []
    
    for j in range(len(z)):
        
        rtsz_maps = []
        picked_y_map = y_map[j]
        
        for i in range(len(nu)):
         
            rtsz_map = rtsz_spec[i] * picked_y_map * (t_e[j]/511) * t_cmb * 1e6
            rtsz_maps.append(rtsz_map)
            
        total_rtsz_maps.append(rtsz_maps)
    
    return np.array(total_rtsz_maps)

In [ ]:
def calculate_tsz_rtsz_map(nu, z, tsz_spec, rtsz_spec, y_map, mu, t_e, t_cmb):
    
    total_tsz_rtsz_maps = []
    
    for j in range(len(z)):
        
        tsz_rtsz_maps = []
        picked_y_map = y_map[j]
        
        for i in range(len(nu)):
             
            tsz_map = tsz_spec[i] * t_cmb * 1e6 * picked_y_map    
            rtsz_map = rtsz_spec[i] * picked_y_map * (t_e[j]/511) * t_cmb * 1e6
            tsz_rtsz_map = tsz_map + rtsz_map
            tsz_rtsz_maps.append(tsz_rtsz_map)
            
        total_tsz_rtsz_maps.append(tsz_rtsz_maps)
    
    return np.array(total_tsz_rtsz_maps)

# rtSZ + noise + CMB map for CMBS4 and CORE frequencies

In [ ]:
def calculate_total_rtsz_map( N, pix_size, nu, cmb_map, rtsz_signal, noise_map):
    
    total_rtsz_maps = []

    for i in range(len(nu)):
        
        total_rtsz_map =  rtsz_signal[i] + noise_map[i]  + cmb_map 
        total_rtsz_maps.append(total_rtsz_map)
        
    return total_rtsz_maps



# All signal map 

In [ ]:
def calculate_all_signal_maps(signal_1, signal_2, signal_3, signal_4, signal_5, z, noise_maps, nu):
    
    total_maps = []
    
    for j in range(len(z)):
    
        all_maps_each_channel = []
        
        map_1 = signal_1[j]
        map_2 = signal_2[j]
        map_3 = signal_3[j]
        map_4 = signal_4[j]
        map_5 = signal_5[j]
        map_6 = noise_maps[j]
    
    
        for i in range(len(nu)):
            
            each_channel_map_2 = map_2[i]
            each_channel_map_3 = map_3[i]
            each_channel_map_4 = map_4[i]
            each_channel_map_5 = map_5[i]
            each_channel_map_6 = map_6[i]
            

            all_map = ( map_1 + each_channel_map_2 + each_channel_map_3 + each_channel_map_4 + 
                       each_channel_map_5 + each_channel_map_6 )

            all_maps_each_channel.append(all_map)
            
        total_maps.append(all_maps_each_channel)
    
    return np.array(total_maps)


######### filtered CMB+kSZ + rkSZ + tkSZ + rtSZ + channel noises map at different CMBS4 frequencies ###########


def calculate_all_signal_filtered_maps(filtered_signal_1, signal_2, signal_3, signal_4, signal_5, noise, nu):
    
    all_filtered_maps = []
    
    for i in range(len(nu)):

        all_filtered_map = filtered_signal_1 + signal_2[i] + signal_3[i] + signal_4[i] + signal_5[i] + noise[i]
        all_filtered_maps.append(all_filtered_map)
    
    return all_filtered_maps



In [ ]:
def all_maps_stacking(z, nu,  all_maps):
    
    total_maps = []

    for i in range(len(nu)):
        
        each_channel_map = []
        np.array(each_channel_map).shape
        
        for j in range(len(z)):
        
             each_map = all_maps[j][i]
             each_channel_map.append(each_map)
            
        total_maps.append(each_channel_map)
        
    stacked_maps = []
        
    for k in range(len(nu)):
    
        stacked_map = np.sum(total_maps[k], axis = 0)/len(z)
        stacked_maps.append(stacked_map)
        
    return np.array(stacked_maps)

# Defining ILC function

In [ ]:

####### Defining ILC function ########

def ilc_run(image,a,factor):

    nf = np.array(image).shape[0]
    npix = np.array(image).shape[1]
    
    image_reshaped = np.array(image).reshape(nf, npix*npix)
    cov_image = np.cov(image_reshaped)
    corr_image = np.corrcoef(image_reshaped)
    # print(corr_image,'\n')
#     plt.imshow(corr_image, cmap = 'jet', origin = 'lower')
#     plt.colorbar()
#     plt.gcf().set_size_inches(8,8)
#     plt.show()
    cov_inverse = np.linalg.inv(cov_image)

    
    weights = (cov_inverse @  a.T) / (a.T @ cov_inverse @ a)
    #weights = (a.T @ cov_inverse) / (a.T @ cov_inverse @ a)

    print("weights =",weights,'\n')
    print("sum weights X a =",np.sum(weights * a),'\n')
          
    reconstructed_map = np.zeros(image[0].shape)
    
    for i in range(nf):
    
        reconstructed_map = reconstructed_map +  (weights[i] * image[i])  / factor 
        #plt.subplot(2, 2, i + 1)
        #plot_map(12,10,f"weight temperature{str(nu_cmbs4[i])} GHz",reconstructed_map,"tau")
        
    return reconstructed_map


In [ ]:
def ilc_weights(image,a):

    nf = np.array(image).shape[0]
    npix = np.array(image).shape[1]
    
    image_reshaped = np.array(image).reshape(nf, npix*npix)
    cov_image = np.cov(image_reshaped)
    corr_image = np.corrcoef(image_reshaped)
    cov_inverse = np.linalg.inv(cov_image)

    weights = (cov_inverse @  a.T) / (a.T @ cov_inverse @ a)
    #weights = (a.T @ cov_inverse) / (a.T @ cov_inverse @ a)

    print("weights =",weights,'\n')
    print("sum weights X a =",np.sum(weights * a),'\n')
    
    return weights

In [ ]:
def ilc_noise(weights, noise_maps, nu):
    
    N = noise_maps.shape[1]

    reconstructed_map = np.zeros((N,N))
        
    for i in range(len(nu)):
    
        reconstructed_map = reconstructed_map +  (weights[i] * np.array(noise_maps)[i])
    
    return reconstructed_map


In [ ]:
def ilc_cmb(weights, cmb_maps, nu):

    reconstructed_map = np.zeros((N,N))
        
    for i in range(len(nu)):
    
        reconstructed_map = reconstructed_map +  (weights[i] * np.array(cmb_maps))
    
    return reconstructed_map


In [ ]:
def ilc_noise_all_signals(signal_1, signal_2, signal_3, signal_4, weights, nu):
    
    N = signal_1.shape[1]
    
    reconstructed_map = np.zeros((N,N))
    
    for i in range(len(nu)):
        
        signal_noise = weights[i] * (signal_1 + signal_2[i] + signal_3[i] + signal_4[i])
        reconstructed_map = reconstructed_map + signal_noise
        
    return reconstructed_map


In [ ]:



########## All signal noise map : ilc weights * (tsz + rksz + tksz + rtsz ) ###########


def ilc_noise_all_signals_ksz(signal_1, signal_2, signal_3, signal_4, weights, nu):
    
    reconstructed_map = np.zeros((N,N))
    
    for i in range(len(nu)):
        
        signal_noise = weights[i] * (signal_1[i] + signal_2[i] + signal_3[i] + signal_4[i])
        reconstructed_map = reconstructed_map + signal_noise
        
    return reconstructed_map


# Defining CILC function

In [ ]:
def cilc_run_1(image,a,b):

    nf = np.array(image).shape[0]
    npix = np.array(image).shape[1]
    
    image_reshaped = np.array(image).reshape(nf, npix*npix)
    cov_image = np.cov(image_reshaped)
    corr_image = np.corrcoef(image_reshaped)
    # print(corr_image,'\n')
#     plt.imshow(corr_image, cmap = 'jet', origin = 'lower')
#     plt.colorbar()
#     plt.gcf().set_size_inches(8,8)
#     plt.show()
    cov_inverse = np.linalg.inv(cov_image)

    weights = (((b.T @ cov_inverse @ b) * (a.T @ cov_inverse)) - ((a.T @ cov_inverse @ b)*(b.T @ cov_inverse))) / ((a.T @ cov_inverse @ a)*(b.T@cov_inverse@b) - (a.T @ cov_inverse @ b) **2)
    
#     p1 = (np.transpose(mix_vect_b) @ inv_cov @ mix_vect_b)*(np.transpose(mix_vect_a) @ inv_cov)
#     p2 = (np.transpose(mix_vect_a) @ inv_cov @ mix_vect_b)*(np.transpose(mix_vect_b) @ inv_cov)
#     p3 = (np.transpose(mix_vect_a) @ inv_cov  @ mix_vect_a) * (np.transpose(mix_vect_b) @ inv_cov  @ mix_vect_b)
#     p4 = (np.transpose(mix_vect_a) @ inv_cov  @ mix_vect_b)**2

    print("weights =",weights,'\n')
    #print("sum weights X a =",np.sum(weights * a),'\n')
    #print("sum weights X b =",np.sum(weights * b),'\n')
          
    reconstructed_map = np.zeros(image[0].shape)
    
    for i in range(nf):
    
        reconstructed_map = reconstructed_map +  weights[i] * image[i]   
        #plt.subplot(2, 2, i + 1)
        #plot_map(12,10,f"weight temperature{str(nu_cmbs4[i])} GHz",reconstructed_map,"tau")
        
    return reconstructed_map


In [ ]:

def cilc_run_2(image,A,e):
    
    nf = np.array(image).shape[0]
    npix = np.array(image).shape[1]
    
    image_reshaped = np.array(image).reshape(nf, npix*npix)
    cov_image = np.cov(image_reshaped)
    corr_image = np.corrcoef(image_reshaped)
    # print(corr_image,'\n')
#     plt.imshow(corr_image, cmap = 'jet', origin = 'lower')
#     plt.colorbar()
#     plt.gcf().set_size_inches(8,8)
#     plt.show()
    cov_inverse = np.linalg.inv(cov_image)

    weights = np.transpose(e)@(np.linalg.inv(A@cov_inverse@np.transpose(A)))@A@cov_inverse
    print("weights = ", weights)
    print(np.sum(weights*A[1]))


    reconstructed_map = np.zeros(image[0].shape)

    for i in range(nf):
    
        reconstructed_map = reconstructed_map +  weights[i] * image[i]   
        #plt.subplot(2, 2, i + 1)
        #plot_map(12,10,f"weight temperature{str(nu_cmbs4[i])} GHz",reconstructed_map,"tau")
        
    return reconstructed_map


In [ ]:
def cilc_weights_1(image, a, b):

    nf = np.array(image).shape[0]
    npix = np.array(image).shape[1]
    
    image_reshaped = np.array(image).reshape(nf, npix*npix)
    cov_image = np.cov(image_reshaped)
    corr_image = np.corrcoef(image_reshaped)

    cov_inverse = np.linalg.inv(cov_image)

    weights = (((b.T @ cov_inverse @ b) * (a.T @ cov_inverse)) - ((a.T @ cov_inverse @ b)*(b.T @ cov_inverse))) / ((a.T @ cov_inverse @ a)*(b.T@cov_inverse@b) - (a.T @ cov_inverse @ b) **2)
    
    print("Weights = ", weights)
    print("sum weights X a =",np.sum(weights * a),'\n')
    print("sum weights X b =",np.sum(weights * b),'\n')
    
    return weights

In [ ]:
def cilc_weights_2(image, A, e):

    nf = np.array(image).shape[0]
    npix = np.array(image).shape[1]
    
    image_reshaped = np.array(image).reshape(nf, npix*npix)
    cov_image = np.cov(image_reshaped)
    corr_image = np.corrcoef(image_reshaped)

    cov_inverse = np.linalg.inv(cov_image)

    weights = np.transpose(e)@(np.linalg.inv(A@cov_inverse@np.transpose(A)))@A@cov_inverse
    
    print("Weights = ", weights)
    print("Weights X a = ", np.sum(weights*A[1]))
    
    return weights

In [ ]:
def cilc_noise(signal_1, signal_2, signal_3, noises, weights, nu):
    
    N = signal_1.shape[1]
    
    reconstructed_map = np.zeros((N,N))
    
    for i in range(len(nu)):
        
        cilc_signal_noise = weights[i] * (signal_1[i] + signal_2[i] + signal_3[i] + noises[i] )
        reconstructed_map = reconstructed_map + cilc_signal_noise
        
    return reconstructed_map


In [ ]:
def cilc_cmb(weights, cmb_maps, nu):
    
    N = cmb_maps.shape[1]

    reconstructed_map = np.zeros((N,N))
        
    for i in range(len(nu)):
    
        reconstructed_map = reconstructed_map +  (weights[i] * np.array(cmb_maps))
    
    return reconstructed_map


# Defining radial profile function

In [ ]:
def dist(image, pix_size):
    
    nx, ny = image.shape[0], image.shape[1]
    center = (nx//2, ny//2)
    YY, XX = np.indices((nx,ny))
    Ra = np.sqrt((XX-center[0])**2 + (YY-center[1])**2) * pix_size

    return(Ra)

#R = dist(extracted_tau_signal_ksz_cmbs4,0.25)
# print(np.max(R),'\n')

def radial_profile(image, n_bins, pix_size):
    
    Ra = dist(image, pix_size)
    profiles = np.zeros((n_bins))
    bin_size = np.linspace(0,np.max(Ra),n_bins+1)

    for i in np.arange(n_bins):
        
        indices = (Ra >= bin_size[i]) & (Ra < bin_size[i+1])
        signals = image[indices]
        profiles[i] = np.mean(signals)
         
    return profiles


def radial_array(n_bins, image, pix_size):
    
    Ra = dist(image, pix_size)
    rad_arr = np.linspace(0,np.max(Ra),n_bins+1) 

    return rad_arr


# Noise residual bin to bin covariance function¶

In [ ]:
def bin_to_bin_covariance(residual_noise_realisations, no_of_realisations, pix_size, n_bins):
    
    nx, ny = np.array(residual_noise_realisations).shape[1], np.array(residual_noise_realisations).shape[2]
    center = (nx//2, ny//2)
    YY, XX = np.indices((nx,ny))
    R = np.sqrt((XX-center[0])**2 + (YY-center[1])**2) * pix_size

    profiles = np.zeros((n_bins,no_of_realisations))
    bin_width = np.linspace(0,np.max(R),n_bins+1)

    for k in np.arange(no_of_realisations):
        
        each_residual_map = np.copy(residual_noise_realisations[k]) # take each map from the array
       
        for j in range(n_bins):
    
            mask = (R >= bin_width[j]) & (R < bin_width[j+1])
            signals = each_residual_map[mask]
            profiles[j,k] = np.mean(signals)
            

    bin_cov = np.cov(profiles)
    
    return bin_cov

# Error calculation of radial profiles of different SZ signals by bin to bin noise covariance

## kSZ and tau profile with errors

 Errors = CMB + all signals + instrumental noise

## Defining functions for error realizations

1) CMB and noise realisations 
2) filtered CMB and noise realisations
3) CMB, other SZ signals and noise realisations (ILC)
4) filtered CMB, other SZ signals and noise realisations (ILC)
5) CMB, other SZ signals and noise realisations (CILC)
6) filtered CMB, other SZ signals and noise realisations (ILC)
7) CMB, other SZ signals and noise realisations (gILC)
8) filtered CMB, other SZ signals and noise realisations (gILC)


In [ ]:

def test_error_realisations_ksz(no_of_realisations, ilc_weights, nu, fwhm, noises):

    error_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        reduced_noise_maps = np.array(noise_maps)/10
        cmb_temp_map = make_CMB_T_map(N,0.25,ell,DlTT)/10
        reduced_cmb_temp_map = cmb_temp_map/10
        
        smoothed_noise_maps = calculate_smoothed_maps(N, pix_size, fwhm, reduced_noise_maps )\
        
        smoothed_cmb_maps = ndimage.gaussian_filter(reduced_cmb_temp_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
    
    
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu)
    
        residual_cmb = smoothed_cmb_maps
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations

In [ ]:

########## Defining functions for calculation of residual errors CMB + channel noises ##############

def error_realisations_ksz(no_of_realisations, no_of_stacking, ilc_weights, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_map = cmb_stacking(no_of_stacking)
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_cmb_maps = ndimage.gaussian_filter(stacked_cmb_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_stacked_noise_maps, nu)
    
        residual_cmb = smoothed_stacked_cmb_maps
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations


In [ ]:

########## Defining functions for calculation of residual errors filtered CMB + channel noises ##############

def error_realisations_ksz_filtered(no_of_realisations, no_of_stacking, ilc_weights_filtered, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_filtered = cmb_filtered_stacking(no_of_stacking)
        
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_filtered_cmb_maps = ndimage.gaussian_filter(stacked_cmb_filtered, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
 
        residual_noise = ilc_noise(ilc_weights_filtered, smoothed_stacked_noise_maps, nu)
    
        residual_cmb = smoothed_stacked_filtered_cmb_maps
    
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations


In [ ]:

########## Defining functions for calculation of residual errors CMB + all signals + channel noises ##############

def error_realisations_ksz_all_signals(no_of_realisations, no_of_stacking, ilc_weights, signal_noises, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_map = cmb_stacking(no_of_stacking)
        
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps(N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_cmb_maps = ndimage.gaussian_filter(stacked_cmb_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
 

        residual_noise = ilc_noise(ilc_weights, smoothed_stacked_noise_maps, nu) + signal_noises
    
        residual_cmb = smoothed_stacked_cmb_maps
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations


######## with no stacking 

def error_realisations_ksz_all_signals_not_stacked(no_of_realisations, ilc_weights, signal_noises, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        
        
        smoothed_noise_maps = calculate_smoothed_maps(N, pix_size, fwhm, noise_maps )
        
        smoothed_cmb_maps = ndimage.gaussian_filter(cmb_map, 
                                                    sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
 
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu) + signal_noises
    
        residual_cmb = smoothed_cmb_maps
        
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations




In [ ]:

########## Defining functions for calculation of residual errors filtered CMB + all signals + channel noises ##############

def error_realisations_ksz_all_signals_filtered(no_of_realisations, no_of_stacking, ilc_weights_filtered, signal_noises, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_filtered = cmb_filtered_stacking(no_of_stacking)
        
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_filtered_cmb_maps = ndimage.gaussian_filter(stacked_cmb_filtered, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
 
        residual_noise = ilc_noise(ilc_weights_filtered, smoothed_stacked_noise_maps, nu) + signal_noises
    
        residual_cmb = smoothed_stacked_filtered_cmb_maps

        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations


####### without stacking

def error_realisations_ksz_all_signals_filtered_not_stacked(no_of_realisations, ilc_weights, signal_noises, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        cmb_map_filtered = poly_filtering(cmb_map, 2, N)
        
        smoothed_noise_maps = calculate_smoothed_maps(N, pix_size, fwhm, noise_maps )
        
        smoothed_filtered_cmb_maps = ndimage.gaussian_filter(cmb_map_filtered, 
                                                    sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu) + signal_noises
    
        residual_cmb = smoothed_filtered_cmb_maps
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations



In [ ]:

########## Defining functions for calculation of residual errors CMB + all signals + channel noises by CILC weights ##############

def error_realisations_cilc_ksz(no_of_realisations, no_of_stacking, signal_1, signal_2, signal_3, nu, fwhm, noises, cilc_weights):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_map = cmb_stacking(no_of_stacking)
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps)
        
        smoothed_stacked_cmb_maps = ndimage.gaussian_filter(stacked_cmb_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        cilc_noises = cilc_noise(signal_1, signal_2, signal_3, smoothed_stacked_noise_maps, cilc_weights, nu_cmbs4)
        
        cilc_cmb_res = smoothed_stacked_cmb_maps
        
        total_residual_map = (cilc_noises + cilc_cmb_res)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations

#### Not stacked

def error_realisations_cilc_ksz_not_stacked(no_of_realisations, signal_1, signal_2, signal_3, nu, fwhm, noises, cilc_weights):

    error_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        
        smoothed_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, noise_maps)
        
        smoothed_cmb_maps = ndimage.gaussian_filter(cmb_map, 
                                                        sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        cilc_noises = cilc_noise(signal_1, signal_2, signal_3, smoothed_noise_maps, cilc_weights, nu_cmbs4)
        
        cilc_cmb_res = smoothed_cmb_maps
        
        total_residual_map = (cilc_noises + cilc_cmb_res)
        error_realisations.append(total_residual_map)
        
    return error_realisations


In [ ]:

########## Defining functions for calculation of residual errors filtered CMB + all signals + channel noises by CILC weights ##############

def error_realisations_cilc_ksz_filtered(no_of_realisations, no_of_stacking, signal_1, signal_2, signal_3, nu, fwhm, noises, cilc_weights_filtered):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        stacked_cmb_filtered = cmb_filtered_stacking(no_of_stacking)
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps)
        
        smoothed_stacked_filtered_cmb_maps = ndimage.gaussian_filter(stacked_cmb_filtered, 
                                                                     sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        
        cilc_noises = cilc_noise(signal_1, signal_2, signal_3, smoothed_stacked_noise_maps, cilc_weights_filtered, nu_cmbs4)
        
        cmb_res = smoothed_stacked_filtered_cmb_maps
        
        total_residual_map = (cilc_noises + cmb_res)
        error_realisations.append(total_residual_map)
        
    return error_realisations


#### Not stacked

def error_realisations_cilc_ksz_filtered_not_stacked(no_of_realisations, signal_1, signal_2, signal_3, nu, fwhm, noises, cilc_weights):

    error_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        cmb_map_filtered = poly_filtering(cmb_map, 2, N)
        
        
        smoothed_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, noise_maps)
        
        smoothed_filtered_cmb_maps = ndimage.gaussian_filter(cmb_map_filtered, 
                                                             sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        cilc_noises = cilc_noise(signal_1, signal_2, signal_3, smoothed_noise_maps, cilc_weights, nu_cmbs4)
        
        cilc_cmb_res = smoothed_filtered_cmb_maps
        
        total_residual_map = (cilc_noises + cilc_cmb_res)
        error_realisations.append(total_residual_map)
        
    return error_realisations



In [ ]:
########## Defining functions for calculation of residual errors =  CMB + rksz + tksz + channel noises by gCILC weights ##############

def error_realisations_gilc_ksz(no_of_realisations, no_of_stacking, signal_1, signal_2, signal_3,nu, fwhm, noises, gcilc_weights):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        stacked_cmb_map = cmb_stacking(no_of_stacking)
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps)
        
        smoothed_stacked_cmb_maps = ndimage.gaussian_filter(stacked_cmb_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        gcilc_noises = cilc_noise(signal_1, signal_2, signal_3, smoothed_stacked_noise_maps, gcilc_weights, nu_cmbs4)
        
        gcilc_cmb_res = smoothed_stacked_cmb_maps
        
        total_residual_map = (gcilc_noises + gcilc_cmb_res)
        error_realisations.append(total_residual_map)
        
    return error_realisations

#### Not stacked

def error_realisations_gilc_ksz_not_stacked(no_of_realisations, signal_1, signal_2, signal_3, nu, fwhm, noises, gcilc_weights):

    error_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        
        smoothed_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, noise_maps)
        
        smoothed_cmb_maps = ndimage.gaussian_filter(cmb_map, 
                                                        sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        gcilc_noises = cilc_noise(signal_1, signal_2, signal_3, smoothed_noise_maps, gcilc_weights, nu_cmbs4)
        
        gcilc_cmb_res = smoothed_cmb_maps
        
        total_residual_map = (gcilc_noises + gcilc_cmb_res)
        error_realisations.append(total_residual_map)
        
    return error_realisations

In [ ]:
########## Defining functions for calculation of residual errors filtered CMB + all signals + channel noises by CILC weights ##############

def error_realisations_gilc_ksz_filtered(no_of_realisations, no_of_stacking, signal_1, signal_2, signal_3, nu, fwhm, noises, gilc_weights_filtered):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        stacked_cmb_filtered = cmb_filtered_stacking(no_of_stacking)
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps)
        
        smoothed_stacked_filtered_cmb_maps = ndimage.gaussian_filter(stacked_cmb_filtered, 
                                                                     sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        
        gcilc_noises = cilc_noise(signal_1, signal_2, signal_3, smoothed_stacked_noise_maps, gilc_weights_filtered, nu_cmbs4)
        
        cmb_res = smoothed_stacked_filtered_cmb_maps
        
        total_residual_map = (gcilc_noises + cmb_res)
        error_realisations.append(total_residual_map)
        
    return error_realisations


#### Not stacked

def error_realisations_gilc_ksz_filtered_not_stacked(no_of_realisations, signal_1, signal_2, signal_3, nu, fwhm, noises, gilc_weights):

    error_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        cmb_map_filtered = poly_filtering(cmb_map, 2, N)
        
        
        smoothed_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, noise_maps)
        
        smoothed_filtered_cmb_maps = ndimage.gaussian_filter(cmb_map_filtered, 
                                                             sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        gcilc_noises = cilc_noise(signal_1, signal_2, signal_3, smoothed_noise_maps, gilc_weights, nu_cmbs4)
        
        gcilc_cmb_res = smoothed_filtered_cmb_maps
        
        total_residual_map = (gcilc_noises + gcilc_cmb_res)
        error_realisations.append(total_residual_map)
        
    return error_realisations

## Calculating the noise covariance by bin to bin covariance matrix

In [ ]:

########## Defining error covariance matrix #############


def noise_covariance(residual_noise_realisations,no_of_realisations, pix_size, n_bins ):
    
    error_covariance = bin_to_bin_covariance(residual_noise_realisations, no_of_realisations, pix_size, n_bins)
    error_variance = np.diagonal(error_covariance)
    sd_error = np.sqrt(error_variance)
    
    return sd_error, error_covariance
    


# Error calculation by noise and CMB realisations tsz

In [ ]:

def error_realisations_tsz_no_stacking(no_of_realisations, ilc_weights, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        
        smoothed_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, noise_maps )
        
        smoothed_cmb_maps = ndimage.gaussian_filter(cmb_map, sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu)
    
        residual_cmb = ilc_cmb(ilc_weights, smoothed_cmb_maps, nu)
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
           
    return error_realisations

In [ ]:

def error_realisations_tsz(no_of_realisations, no_of_stacking, ilc_weights, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        stacked_cmb_map = cmb_stacking(no_of_stacking)
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_cmb_maps = ndimage.gaussian_filter(stacked_cmb_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_stacked_noise_maps, nu)
    
        residual_cmb = ilc_cmb(ilc_weights, smoothed_stacked_cmb_maps, nu)
    
        total_residual_map = (residual_noise + residual_cmb)
        error_realisations.append(total_residual_map)
           
    return error_realisations

In [ ]:
########## Defining functions for calculation of residual errors CMB + all signals + channel noises ##############

def error_realisations_tsz_all_signals(no_of_realisations, ilc_weights, signal_1, signal_2, signal_3, nu, fwhm, noises, v, mu, t_cmb):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        cmb_ksz_map = calculate_cmb_ksz_map(cmb_map, v, mu, t_cmb)
        
        smoothed_noise_maps = calculate_smoothed_maps(N, pix_size, fwhm, noise_maps )
        
        
        smoothed_cmb_ksz_maps = ndimage.gaussian_filter(cmb_ksz_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        signal_noises = ilc_noise_all_signals(smoothed_cmb_ksz_maps, signal_1, signal_2, signal_3, ilc_weights, nu)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu) + signal_noises
    
    
        total_residual_map = residual_noise
        error_realisations.append(total_residual_map)
        
    return error_realisations


In [ ]:

########## Defining functions for calculation of residual errors by extracting tSZ from all signals + channel noises by CILC weights ##############

def error_realisations_cilc_tsz(no_of_realisations, signal_1, signal_2, signal_3, nu, fwhm, noises, cilc_weights):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        
        smoothed_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, noise_maps)
        
        cilc_noises = cilc_noise(signal_1, signal_2, signal_3, smoothed_noise_maps, cilc_weights, nu_cmbs4)
        
        total_residual_map = cilc_noises 
        
        error_realisations.append(total_residual_map)
        
    return error_realisations

In [ ]:
def error_realisations_tsz_all_signals_stacked(no_of_realisations, no_of_stacking, ilc_weights, signal_1, signal_2, signal_3, vel, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_ksz_map = calculate_cmb_ksz_stacked_map(no_of_realisations, vel)
        
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_cmb_ksz_maps = ndimage.gaussian_filter(stacked_cmb_ksz_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        signal_noises = ilc_noise_all_signals(smoothed_stacked_cmb_ksz_maps, signal_1, signal_2, signal_3, ilc_weights, nu)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_stacked_noise_maps, nu) + signal_noises
    
        total_residual_map = residual_noise
        
        error_realisations.append(total_residual_map)
           
    return error_realisations


In [ ]:
def error_realisations_cilc_tsz_stacked(no_of_realisations, no_of_stacking, signal_1, signal_2, signal_3, nu, fwhm, noises, cilc_weights):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps)
        
        cilc_noises = cilc_noise(signal_1, signal_2, signal_3, smoothed_stacked_noise_maps, cilc_weights, nu_cmbs4)
        
        
        total_residual_map = (cilc_noises)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations

# rksz noise and CMB realisations

In [ ]:
def error_realisations_rksz_not_stacked(no_of_realisations, ilc_weights, nu, fwhm, noises):

    error_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        
        smoothed_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, noise_maps )
        
        smoothed_cmb_maps = ndimage.gaussian_filter(cmb_map, 
                                                    sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu)
    
        residual_cmb = ilc_cmb(ilc_weights, smoothed_cmb_maps, nu)
        
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations

In [ ]:

def error_realisations_rksz(no_of_realisations, no_of_stacking, ilc_weights, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_map = cmb_stacking(no_of_stacking)
        
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_cmb_maps = ndimage.gaussian_filter(stacked_cmb_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
    
 
        residual_noise = ilc_noise(ilc_weights, smoothed_stacked_noise_maps, nu)
    
        residual_cmb = ilc_cmb(ilc_weights, smoothed_stacked_cmb_maps, nu)
        
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
           
    return error_realisations

In [ ]:

def error_realisations_rksz_all_signals(no_of_realisations, no_of_stacking, ilc_weights, signal_1, signal_2, signal_3, vel, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_ksz_map = calculate_cmb_ksz_stacked_map(no_of_realisations, vel)
        
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_cmb_ksz_maps = ndimage.gaussian_filter(stacked_cmb_ksz_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        
        signal_noises = ilc_noise_all_signals(smoothed_stacked_cmb_ksz_maps, signal_1, signal_2, signal_3, ilc_weights, nu)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_stacked_noise_maps, nu) + signal_noises
    
    
        total_residual_map = residual_noise
        
        error_realisations.append(total_residual_map)
           
    return error_realisations



In [ ]:
def ilc_error_realisations_rksz(no_of_realisations,ilc_weights,tau_map, pix_size, z, v, nu, fwhm, noises, ell, DlTT, mu, t_cmb, signal_1, signal_2, signal_3):
    
    N =  signal_1.shape[1]
    
    error_realisations = []
    
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps(N, z, pix_size, noises)
        
        cmb_ksz_maps = calculate_cmb_ksz_map(tau_map,z, v, N , pix_size, ell, DlTT, mu, t_cmb )
        
        
        noise_maps_smoothed = calculate_smoothed_maps(z, pix_size, fwhm, noise_maps)
        
        cmb_ksz_maps_smoothed = calculate_smoothed_maps_freq_independent(z, pix_size, fwhm, cmb_ksz_maps)
        
        
        noise_maps_smoothed_stacked = all_maps_stacking(z, nu, noise_maps_smoothed)
        
        cmb_ksz_maps_smoothed_stacked =  np.sum(cmb_ksz_maps_smoothed, axis = 0)/len(z)
        
        
        signal_noises = ilc_noise_all_signals(cmb_ksz_maps_smoothed_stacked, signal_1, signal_2, signal_3, ilc_weights, nu)
 
        instrumental_noises = ilc_noise(ilc_weights, noise_maps_smoothed_stacked, nu)
    
    
        total_residual_noises = signal_noises + instrumental_noises
        
        error_realisations.append(total_residual_noises)
           
    return np.array(error_realisations)

In [ ]:


def error_realisations_rksz_all_signals_not_stacked(no_of_realisations, ilc_weights, signal_1, signal_2, signal_3, v, mu, t_cmb, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        
        cmb_ksz_map = calculate_cmb_ksz_map(cmb_map, v, mu, t_cmb)
        
        
        smoothed_noise_maps = calculate_smoothed_maps(N, pix_size, fwhm, noise_maps )
        
        smoothed_cmb_ksz_maps = ndimage.gaussian_filter(cmb_ksz_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        
        signal_noises = ilc_noise_all_signals(smoothed_cmb_ksz_maps, signal_1, signal_2, signal_3, ilc_weights, nu)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu) + signal_noises
    
    
        total_residual_map = residual_noise
        
        error_realisations.append(total_residual_map)
        
    return error_realisations

In [ ]:
def cilc_error_realisations_rksz(no_of_realisations, pix_size, z, nu, fwhm, noises, cilc_weights, signal_1, signal_2, signal_3):
    
    N = signal_1.shape[1]
    
    error_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps(N, z, pix_size, noises)
        
        noise_maps_smoothed = calculate_smoothed_maps(z, pix_size, fwhm, noise_maps)
        
        noise_maps_smoothed_stacked = all_maps_stacking(z, nu, noise_maps_smoothed)
    
    
        cilc_noises = cilc_noise(signal_1, signal_2, signal_3, noise_maps_smoothed_stacked, cilc_weights, nu)
        
        error_realisations.append(cilc_noises)
        
    return np.array(error_realisations)


# tksz noise and CMB realisations

In [ ]:
def error_realisations_tksz_not_stacked(no_of_realisations, ilc_weights, nu, fwhm, noises):

    error_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        
        smoothed_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, noise_maps )
        
        smoothed_cmb_maps = ndimage.gaussian_filter(cmb_map, 
                                                    sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu)
    
        residual_cmb = ilc_cmb(ilc_weights, smoothed_cmb_maps, nu)
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations

In [ ]:
def error_realisations_tksz(no_of_realisations, no_of_stacking, ilc_weights, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_map = cmb_stacking(no_of_stacking)
        
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_cmb_maps = ndimage.gaussian_filter(stacked_cmb_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
 
        residual_noise = ilc_noise(ilc_weights, smoothed_stacked_noise_maps, nu)
    
        residual_cmb = ilc_cmb(ilc_weights, smoothed_stacked_cmb_maps, nu)
        
    
        total_residual_map = (residual_noise + residual_cmb)
        
        error_realisations.append(total_residual_map)
           
    return error_realisations

In [ ]:

def error_realisations_tksz_all_signals(no_of_realisations, no_of_stacking, ilc_weights, signal_1, signal_2, signal_3, vel, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_ksz_map = calculate_cmb_ksz_stacked_map(no_of_realisations, vel)
        
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_cmb_ksz_maps = ndimage.gaussian_filter(stacked_cmb_ksz_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        
        signal_noises = ilc_noise_all_signals(smoothed_stacked_cmb_ksz_maps, signal_1, signal_2, signal_3, ilc_weights, nu)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_stacked_noise_maps, nu) + signal_noises
    
    
        total_residual_map = residual_noise
        
        error_realisations.append(total_residual_map)
           
    return error_realisations



In [ ]:
def ilc_error_realisations_tksz(no_of_realisations,ilc_weights,tau_map, pix_size, z, v, nu, fwhm, noises, ell, DlTT, mu, t_cmb, signal_1, signal_2, signal_3):
    
    N =  signal_1.shape[1]
    
    error_realisations = []
    
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps(N, z, pix_size, noises)
        
        cmb_ksz_maps = calculate_cmb_ksz_map(tau_map, z, v, N, pix_size, ell, DlTT, mu, t_cmb )
        
        
        noise_maps_smoothed = calculate_smoothed_maps(z, pix_size, fwhm, noise_maps)
        
        cmb_ksz_maps_smoothed = calculate_smoothed_maps_freq_independent(z, pix_size, fwhm, cmb_ksz_maps)
        
        
        noise_maps_smoothed_stacked = all_maps_stacking(z, nu, noise_maps_smoothed)
        
        cmb_ksz_maps_smoothed_stacked =  np.sum(cmb_ksz_maps_smoothed, axis = 0)/len(z)
        
        
        signal_noises = ilc_noise_all_signals(cmb_ksz_maps_smoothed_stacked, signal_1, signal_2, signal_3, ilc_weights, nu)
 
        instrumental_noises = ilc_noise(ilc_weights, noise_maps_smoothed_stacked, nu)
    
    
        total_residual_noises  = signal_noises + instrumental_noises
        
        error_realisations.append(total_residual_noises)
           
    return np.array(error_realisations)

In [ ]:


def error_realisations_tksz_all_signals_not_stacked(no_of_realisations, ilc_weights, signal_1, signal_2, signal_3, v, mu, t_cmb, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        
        cmb_ksz_map = calculate_cmb_ksz_map(cmb_map, v, mu, t_cmb)
        
        
        smoothed_noise_maps = calculate_smoothed_maps(N, pix_size, fwhm, noise_maps )
        
        
        smoothed_cmb_ksz_maps = ndimage.gaussian_filter(cmb_ksz_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        
        signal_noises = ilc_noise_all_signals(smoothed_cmb_ksz_maps, signal_1, signal_2, signal_3, ilc_weights, nu)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu) + signal_noises
    
    
        total_residual_map = residual_noise
        
        error_realisations.append(total_residual_map)
        
    return error_realisations

In [ ]:
def cilc_error_realisations_tksz(no_of_realisations, pix_size, z, nu, fwhm, noises, cilc_weights, signal_1, signal_2, signal_3):
    
    N = signal_1.shape[1]
    
    error_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps(N, z, pix_size, noises)
        
        noise_maps_smoothed = calculate_smoothed_maps(z, pix_size, fwhm, noise_maps)
        
        noise_maps_smoothed_stacked = all_maps_stacking(z, nu, noise_maps_smoothed)
    
    
        cilc_noises = cilc_noise(signal_1, signal_2, signal_3, noise_maps_smoothed_stacked, cilc_weights, nu)
        
        error_realisations.append(cilc_noises)
        
    return np.array(error_realisations)

# rtsz noise  and CMB realisations

In [ ]:
def error_realisations_rtsz_not_stacked(no_of_realisations, ilc_weights, nu, fwhm, noises):

    error_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        
        smoothed_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, noise_maps )
        
        smoothed_cmb_maps = ndimage.gaussian_filter(cmb_map, 
                                                    sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu)
    
        residual_cmb = ilc_cmb(ilc_weights, smoothed_cmb_maps, nu)
    
        total_residual_map = (residual_noise + residual_cmb)
        error_realisations.append(total_residual_map)
        
    return error_realisations

In [ ]:
def error_realisations_rtsz(no_of_realisations, no_of_stacking, ilc_weights, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        stacked_cmb_map = cmb_stacking(no_of_stacking)
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_cmb_maps = ndimage.gaussian_filter(stacked_cmb_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_stacked_noise_maps, nu)
    
        residual_cmb = ilc_cmb(ilc_weights, smoothed_stacked_cmb_maps, nu)
    
        total_residual_map = (residual_noise + residual_cmb)
        error_realisations.append(total_residual_map)
           
    return error_realisations

In [ ]:

def error_realisations_rtsz_all_signals(no_of_realisations, no_of_stacking, ilc_weights, signal_1, signal_2, signal_3, vel, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        stacked_noise_maps =  noise_map_stacking(no_of_stacking, noises)
        
        stacked_cmb_ksz_map = calculate_cmb_ksz_stacked_map(no_of_realisations, vel)
        
        smoothed_stacked_noise_maps = calculate_smoothed_maps( N, pix_size, fwhm, stacked_noise_maps )
        
        smoothed_stacked_cmb_ksz_maps = ndimage.gaussian_filter(stacked_cmb_ksz_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        signal_noises = ilc_noise_all_signals(smoothed_stacked_cmb_ksz_maps, signal_1, signal_2, signal_3, ilc_weights, nu)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_stacked_noise_maps, nu) + signal_noises
    
        total_residual_map = residual_noise
        
        error_realisations.append(total_residual_map)
           
    return error_realisations



In [ ]:


def error_realisations_rtsz_all_signals_not_stacked(no_of_realisations, ilc_weights, signal_1, signal_2, signal_3, v, mu, t_cmb, nu, fwhm, noises):

    error_realisations = []
    #noise_realisations = []
    #cmb_realisations = []
  
    for t in range(no_of_realisations):
        
        noise_maps = all_channel_noise_maps( N, pix_size, noises)
        cmb_map = make_CMB_T_map(N,pix_size,ell,DlTT)
        cmb_ksz_map = calculate_cmb_ksz_map(cmb_map, v, mu, t_cmb)
        
        smoothed_noise_maps = calculate_smoothed_maps(N, pix_size, fwhm, noise_maps )
        
        
        smoothed_cmb_ksz_maps = ndimage.gaussian_filter(cmb_ksz_map, 
                                                            sigma = fwhm[0]/(np.sqrt(8*np.log(2)))/pix_size, order = 0, mode = 'reflect', truncate = 10)
        
        signal_noises = ilc_noise_all_signals(smoothed_cmb_ksz_maps, signal_1, signal_2, signal_3, ilc_weights, nu)
 
        residual_noise = ilc_noise(ilc_weights, smoothed_noise_maps, nu) + signal_noises
    
    
        total_residual_map = residual_noise
        
        error_realisations.append(total_residual_map)
        
        error_realisations.append(total_residual_map)
        
    return error_realisations

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy import interpolate
import scipy.integrate as integrate
from astropy.constants import pc, e,c,h,m_e, M_sun, sigma_T
from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import Planck15

from matplotlib import figure
from matplotlib import ticker
from matplotlib.ticker import FixedLocator
from matplotlib.colors import LinearSegmentedColormap, colorConverter
from matplotlib.ticker import MultipleLocator

In [ ]:
def readfits(file_name):
    '''Reads fits files and returns data array and header.

    Parameters
    ----------
    file_name: string
        Name of the fits file

    Returns
    -------
    data: float array
        Data array extracted from the fits file
    header: fits header
        Header of the fits file
    '''

    hdulist=fits.open(file_name)
    data = hdulist[0].data
    header = hdulist[0].header
    hdulist.close()

    return(data, header)


def draw_clusters(n, nclusters_result, z_lower=0.01, z_upper=5, M_500_lower=5e13, M_500_upper=2e15, limit = 4.5):

    n_grid = nclusters_result.shape[0]    

    z_1d = np.geomspace(z_lower, z_upper, n_grid+1)
    z_1d = (np.log10(z_1d)[1:]+np.log10(z_1d)[0:-1])/2
    M_500_1d = np.geomspace(M_500_lower, M_500_upper, n_grid+1)
    M_500_1d = (np.log10(M_500_1d)[1:]+np.log10(M_500_1d)[0:-1])/2 

    tsz_interpol = interpolate.RectBivariateSpline(z_1d, M_500_1d, np.transpose(nclusters_result), kx=1, ky=1)

    sample = np.zeros((2, n)) 
    max_counts = np.max(nclusters_result)
    count = 0

    while count < n:
        rand = np.random.random(size=3)
        z = rand[0] * (np.log10(z_upper)-np.log10(z_lower)) + np.log10(z_lower)
        m = rand[1] * (np.log10(M_500_upper)-np.log10(M_500_lower)) + np.log10(M_500_lower)
        y = rand[2] * max_counts
        value = tsz_interpol(z,m)
        if y <= value:
            sample[:,count] = [10**z,10**m]
            count += 1
            
    return(sample)

In [ ]:
from astropy.constants import pc, e,c,h,m_e,m_p, M_sun, sigma_T
from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import Planck15

pc = pc.si.value
e = e.si.value
c = c.si.value
h = h.si.value
m_e = m_e.si.value
m_p = m_p.si.value
M_sun = M_sun.si.value
sigma_T = sigma_T.si.value
T_CMB = 2.7255

cosmo = FlatLambdaCDM(H0= 70, Om0 = 0.3)
H0 = cosmo.H0.value

In [ ]:
omg_m = 0.3
omg_l = 0.7
#r_c = 2
beta = 0.86
f_gas = 0.125

f_h = 0.76



## $M_{500} = \frac{4\pi}{3}[500\rho_c(z)]r_{500}^3$


In [ ]:

def calculate_m500_to_r500(red_shifts, m_500):

    rho_crit = cosmo.critical_density(red_shifts).si.value     # in kg/m^3
    R_500 = ( m_500*M_sun * 3/(4*np.pi*(500*rho_crit)))**(1./3)      #in meters

    return R_500



def r500_in_arcmins(r_500, redshifts):
    
    return r_500/ (cosmo.angular_diameter_distance(red_shifts).si.value) * 180 / np.pi *60       #convert from m to radians to arcmin




In [ ]:
def calculate_E(z, omg_m, omg_l):
    
    val = []
    
    for i in range(len(z)):
        
        E = omg_m * ((1+z[i])**3) + omg_l
        val.append(E)
        
    
    return val


In [ ]:
def calculate_T_e(m_500,E):
    
    return np.power((m_500*(np.power(E,1.04)))/(1e14 * 0.291),(1/1.62))

    

In [ ]:

def calculate_omega(red_shifts, omg_m, omg_l):
    
    omega_list = []
    
    for i in range(len(red_shifts)):
        
        omega = (omg_m * (1 + np.power(red_shifts[i],2)))/( omg_m * (1 + np.power(red_shifts[i],3) + omg_l))
        omega_list.append(omega)
        
    return np.array(omega_list)



def calculate_del(omega):
    
    return 18*(np.pi**2) + 82*(omega - 1) - 39 * (np.power((omega-1),2))
    





def calculate_m_vir(delta,m_500):
    
    return ((8*delta*m_500)/500)




def calculate_conc_parameter(red_shifts, m_vir):
    
    c_list = []
    
    for i in range(len(red_shifts)):
        
        c = (5.72/((1+red_shifts[i])**0.71)) * ((m_vir[i]/1e14)**(-0.081))
        c_list.append(c)
        
    return(np.array(c_list))


In [ ]:
def r_c (r_500, conc_parameter):
    
    return (2*r_500)/conc_parameter



def r_c_in_arcmins(r_c, red_shifts):
    
    return r_c/ (cosmo.angular_diameter_distance(red_shifts).si.value) * 180 / np.pi *60       #convert from m to radians to arcmin



def c_500 (r_500, r_c):

    return r_500/r_c

In [ ]:
def calculate_f_1(c_500):
    
    integrate_res = []
    
    for i in range(len(c_500)):
        
        integrand = lambda x: 1/((1+x**2)**(1.5*beta))
        
        integrated = integrate.quad(integrand, 0, c_500[i])[0]
        
        integrate_res.append(integrated)
    
    return integrate_res
    

def calculate_f_2(c_500):
    
    integrate_res = []
    
    for i in range(len(c_500)):
        
        integrand = lambda x: ( x**2/(( 1 + x**2 )**(1.5 * beta)))
        
        integrated = integrate.quad(integrand, 0, c_500[i])[0]
        
        integrate_res.append(integrated)
    
    return integrate_res
    

In [ ]:
def calculate_tau(m_500,f_1,f_2,f_h,r_c):
    
    store = []
    
    for i in range(len(m_500)):

        tau = (sigma_T * f_1[i] * (1+f_h) * f_gas * m_500[i])/(4 * np.pi * r_c[i]**2 * f_2[i] * 1.672623099e-57)
        store.append(tau)
    
    return np.array(store) 


In [ ]:
def plot_map_500(x,y,map_title,map_plot,cbar_label, N, pix_size):
    
    plt.gcf().set_size_inches(x,y)

    plt.title(map_title, fontsize=15, fontweight='bold')
    image = plt.imshow(map_plot, cmap = 'jet', origin = 'lower')
    cb = plt.colorbar(image)
    cb.set_label(label = cbar_label, fontsize=15, fontweight='bold')
    cb.ax.tick_params(labelsize=15)
    
    
    X_width = N #* pix_size/60. 
    Y_width = N #* pix_size/60
    
    
    
    image.set_extent([-X_width/20,X_width/20,-Y_width/20,Y_width/20])
    plt.ylabel(r'$\theta_{500}$', fontsize=15)
    plt.xlabel(r'$\theta_{500}$', fontsize=15)
    ax = plt.gca()


    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    ax.spines['top'].set_linewidth(0.8)
    ax.spines['right'].set_linewidth(0.8)
    ax.spines['bottom'].set_linewidth(0.8)
    ax.spines['left'].set_linewidth(0.8)

In [ ]:

def simulate_y_map(npix ,beta, tau, pix_size, t_e, z, r_core ):
    
    y_maps = []
    
    for i in range(len(z)):
    
    
        y_0 = tau[i] * (t_e[i]/511)
        
        YY,XX = np.indices((npix, npix))
        center = (npix//2,npix//2)
        r = np.sqrt((XX-center[0])**2 + (YY-center[1])**2)*pix_size[i]
    
        y = y_0 * (1 + ( r / r_core[i] ) ** 2 ) ** ((1-3 * beta) / 2) 
        
        y_maps.append(y)
    
    return(np.array(y_maps))
    

In [ ]:
def simulate_tau_map(npix ,beta, tau, pix_size, t_e, z, r_core):
    
    
    tau_maps = []
    
    for i in range(len(z)):
    
    
        y_0 = tau[i] * (t_e[i]/511)
        
        YY,XX = np.indices((npix, npix))
        center = (npix//2,npix//2)
        r = np.sqrt((XX-center[0])**2 + (YY-center[1])**2)*pix_size[i]
    
        y = y_0 * (1 + ( r / r_core[i] ) ** 2 ) ** ((1-3 * beta) / 2) 
    
        tau_map = y /(t_e[i]/511)
        
        tau_maps.append(tau_map)
    
    return(np.array(tau_maps))
    
